이름: 이선호

학번: 20195214


## Final project 규칙

- 학생들 간의 질문은 금지합니다 (교수 및 조교에게 질문하세요)
- 대 원칙은 "내가 이해한 것만 활용하고 작성한다" 입니다
- 사용하는 기법들과 코드는 모두 보고서에 설명되어야하며, 설명이 없는 경우 감점이나 0점 처리합니다
    - 예를 들어서 검색한 끝에 model ensemble이라는 기법이 유용할 것 같아서, 참고하여 성능을 개선한 경우
        1. 참고자료의 출처를 작성할 것
        2. 기반 이론을 설명할 것
        3. 사용한 코드 분석 및 기법을 설명할 것
- 주석이 없는 경우 채점하지 않습니다
- Pretrained network은 사용하지 않습니다. 본 프로젝트에서는, random 초기화한 parameter들을 직접 학습하는 과정만 허용합니다
    - 랜덤 초기화 기법은 특별한 제약이 없습니다
- Competition 순위와 보고서는 각각 5:5로 점수가 반영됩니다.

# Final Project 

## Project: 새 품종 분류하기

## Due date: 2021.06.15

https://www.kaggle.com/c/detect-the-bird/leaderboard
---

* 아래 여러 셀에서 코드를 완성하는 부분을 수행하고, 보고서에 설명을 최대한 자세하게 적어주세요. 기준은 본인이 이해하고 있다는 것을 표현할 수 있는 부분을 모두 적으시면 됩니다.
  
  
> **제출방법**: 
* 보고서에는 코드 캡쳐 첨부이외에도, 각 코드를 작성하는 기반 이론, 방법론과 설명을 작성하세요.

### 목표: 앱을 위한 인공지능 알고리즘 개발 
* 본 보고서에서는 모바일/웹앱을 위한 인공지능 알고리즘을 개발하라는 업무를 부여받았다고 가정합니다
* 프로젝트가 완성본은, 사용자가 제공하는 image를 받아서 새의 종(種)을 예측합니다


---

## Step 0: Import library

- 필요하다 생각되는 라이브러리를 미리 import해 놓은 항목입니다.(참고)
- 필요 없는 라이브러리를 제거하거나 필요한 라이브러리를 추가하셔도 됩니다.

In [1]:
import shutil
import os
import glob
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pandas as pd
#import helper

from pathlib import Path
from collections import OrderedDict


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Step 1: Dataset 준비하기

In [ ]:
#코드작성




transform = transforms.Compose([transforms.RandomRotation(10),
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5, 0.5, 0.5],
                                                    [0.5, 0.5, 0.5])
                                ])
#대부분 사진 가운데 새가 배치됨, 224사이즈로 만들기, 테두리 cut, Normalize(0.5), Tensor화 진행
trainset = datasets.ImageFolder('../input/detectthebird/imgs/train', transform=transform)
#ImageFolder를 이용한 dataset준비, trainform을 통한 데이터 전처리


## Step 2: Dataset 에 대한 Data Loaders 구성

In [15]:
#코드작성
valid_size = 0.2  #Dataset을 Train set과 Validation set으로 나눌 비율 설정
batch_size = 64   #Data Loader 구성시 data set의 batch size설정
num_workers =0    # 0개의 subprocesses 가종

num_train = len(trainset)  #trainset전체의 길이(sample 개수)
num_train = int(num_train) #trainset sample 개수 정수화
indices = list(range(num_train)) # 0 ~ sample 개수 -1까지 원소로 갖는 리스트 생성
np.random.shuffle(indices) #리스트 indices내의 원소를 랜덤 셔플
split = int(np.floor(valid_size * num_train)) #dataset을 20%비율로 스플릿 해줄 index 계산
train_idx, valid_idx = indices[split:], indices[:split] 
#계산한 index에 따라 80%와 20%로 dataset을 train set과 validation set으로 나눔

train_sampler = SubsetRandomSampler(train_idx) #랜덤 리스트 생성
valid_sampler = SubsetRandomSampler(valid_idx) #랜덤 리스트 생성

train_loader = DataLoader(trainset, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
#설정한 dataset, batch size, sampler, num_workers값에 따라 train loader 생성
valid_loader = DataLoader(trainset, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
#설정한 dataset, batch size, sampler, num_workers값에 따라 vaild loader 생성


---

## Step 3: Neural Network 생성
- Pretrained model을 허용하지 않습니다. (직접 모델을 설계해 주세요)


In [7]:
#코드작성
#코드작성
from torch import nn, optim
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module): #nn.Module 상속
    def __init__(self):
        super().__init__() #nn.Module.__init__() 실행
        
        self.conv1 = nn.Conv2d(3, 10, 8, padding=1) 
        # kernel = 8x8, 채널 수 10인 Convolution Network
        
        self.pool = nn.MaxPool2d(2,2)
        #Pooling Layer - MaxPooling (2x2) stride =2
        
        
        self.conv2 = nn.Conv2d(10, 20, 3, padding=1)
        # kernel = 3x3, 채널 수 20인 Convolution Network

        self.conv3 = nn.Conv2d(20, 40, 3, padding=1)
        # kernel = 3x3, 채널 수 40인 Convolution Network

        self.conv4 = nn.Conv2d(40, 20, 3, padding=1)
        # kernel = 3x3, 채널 수 20인 Convolution Network
        
        self.fc1 = nn.Linear(14580, 7290)
        #input dimension : 14580 output diemsion : 7290인 Linear Regression model
        
        self.fc2 = nn.Linear(7290, 3145)
        #input dimension : 7290 output diemsion : 3145인 Linear Regression model
        
        self.fc3 = nn.Linear(3145, 15)
        #input dimension : 3145 output diemsion : 15인 Linear Regression model
        
        self.dropout = nn.Dropout(0.25)
        #dropout layer

        
    def forward(self, x):
        #Convolution layer -> Relu -> Maxpool layer적용
        x = self.pool(F.relu(self.conv1(x)))
        #Convolution layer -> Relu -> Maxpool layer적용
        x = self.pool(F.relu(self.conv2(x)))
        #Convolution layer -> Relu -> Maxpool layer적용
        x = self.pool(F.relu(self.conv3(x)))
        #Convolution layer -> Relu 적용
        x = F.relu(self.conv4(x))
        #image input을 평탄화
        x = x.view(x.shape[0], -1)
        #Linear layer -> Relu 적용
        x = F.relu(self.fc1(x))
        #dropout layer 적용
        x = self.dropout(x)
        #Linear layer -> Relu 적용
        x = F.relu(self.fc2(x))
        #dropout 적용
        x = self.dropout(x)
        #Linear layer -> Relu
        x = self.fc3(x)

        return x
#모델 객체 생성
model = Classifier()

---

 ## Step 4: Cost (Loss) Function 과 Optimizer 선택
 
 [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) 및 [optimizer](http://pytorch.org/docs/stable/optim.html)를 선택하여 코드를 완성하세요.
 
 위 링크에서 다양한 Loss Function과 Optimize Function을 확인 할 수 있습니다
 

In [ ]:
#loss function 선택 - CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
#optimizer 선택 - SGD, learning rate : 0.01
optimizer = optim.SGD(model.parameters(), lr=0.01)

#Softmax
softmax = nn.Softmax(dim=1)
#가장 높은 확률 값을 리턴해주는 함수 get_pred생성
def get_pred(ps):
    return torch.argmax(ps, dim=1)

---

## Step 5: 구성한 모델에 대한 Train and Validate 진행

* 코드 전체를 주석으로 설명하세요
* Epoch 별로 Loss나 Accuracy를 출력하여 학습 진행 과정을 확인 할 수 있도록 합니다
* 출력 예시는 주어지나 정해진 형식은 없습니다
* 최적의 모델 저장

예제:
```
Started Training...
Epoch: 1 	Training Loss: 3.317162 	Validation Loss: 4.162958
Epoch: 2 	Training Loss: 2.420140 	Validation Loss: 4.182362
...
...
Finished training
```

In [ ]:
#epochs 설정
epochs = 500
#train set의 loss를 저장할 train_loss
train_loss = torch.zeros(epochs)
#validation set의 loss를 저장할 val_loss
val_loss = torch.zeros(epochs)
#train set의 예측 성공률을 저장할 train_acc
train_acc = torch.zeros(epochs)
#validation set의 예측 성공률을 저장할 val_acc
val_acc = torch.zeros(epochs) 


for e in range(epochs): #epochs 수 만큼 반복
    print(f"------ Starting epoch: {e} --------")

    
    ##########Train set##########
    
    for inputs, labels in train_loader: #train set에 있는 image들과 label을 순서대로
       
        #모델에 image 입력
        logits = model(inputs)
        #loss 계산
        loss = criterion(logits, labels)

        #optimizer 초기화  -> gradients값을 0으로 초기화   
        optimizer.zero_grad()

        #loss에 대한 미분 계산 (backpropagation)
        loss.backward()
        
        #optimizer 진행 Stochastic gradient descent(SGD) 
        optimizer.step()

        #train_loss에 loss값 저장
        train_loss[e] += loss.item()
        #softmax함수를 통해 확률값으로 변환
        ps = softmax(logits)
        #확률값 중 가장 큰 값 저장
        pred = get_pred(ps)
        #예측과 결과가 같은 것들의 평균 계산 - 정확도 계산
        train_acc[e] += torch.sum(pred==labels)/labels.shape[0] 
    #epoch당 loss계산
    train_loss[e] /= len(train_loader)
    #epoch당 예측 정확도계산
    train_acc[e] /= len(train_loader)  
    
    ############Vaildaion set###############
    
    #gradient 계산 중지
    with torch.no_grad():
        #evalutation mode (dropout 비활성화)
        model.eval()
        for inputs, labels in valid_loader:
            
            #모델에 image 입력
            logits = model(inputs)
            #loss 계산
            loss = criterion(logits, labels)

            #train_loss에 loss값 저장
            val_loss[e] += loss.item()
            #softmax함수를 통해 확률값으로 변환            
            ps = softmax(logits)
            #확률값 중 가장 큰 값 저장            
            pred = get_pred(ps)
            #예측과 결과가 같은 것들의 평균 계산 - 정확도 계산
            val_acc[e] += torch.sum(pred==labels)/labels.shape[0] 

        #epoch당 loss계산
        val_loss[e] /= len(valid_loader)
        #epoch당 예측 정확도계산
        val_acc[e] /= len(valid_loader) 
    
     # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss[e], val_loss[e]))
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
        e, train_acc[e], val_acc[e]))

---

## Step 6: CNN model training/validation 분석
   * tranining loss와 validation loss 그래프를 통해서 분석


In [ ]:
#train loss 그래프 출력
plt.plot(train_loss)
#validation loss 그래프 출력
plt.plot(val_loss)
#그래프에 십자선
plt.grid()

* 최적의 모델 로드

In [ ]:
#train 예측정확도 그래프 출력
plt.plot(train_acc)
#validation 예측정확도 그래프 출력
plt.plot(val_acc)
#그래프에 십자선
plt.grid()

---

## Step 7: Predict with Test Data 


### 예시 코드를 제공해 드립니다. 필요한 부분을 채워 사용하시거나 직접 코드를 작성 하셔도 됩니다

In [ ]:
class dataset(Dataset):
    def __init__(self,imgpath,transform=None): #image data set 경로, tranform 입력
        
        self.imgpath = imgpath
        self.transform = transform
        
    def __len__(self):
        #length of data set
        return len(self.imgpath)
    
    def __getitem__(self,idx):
        x = self.transform(Image.open(self.imgpath[idx]).convert('RGB'))
        
        return x

### test dataset과 dataloader 생성


In [19]:
#test data 불러오기
test_set = sorted(glob.glob('../input/detect-the-bird/imgs/test/*'))

#test set을 클래스 dataset을 이용해 transform을 적용
test_data = dataset(test_set,transform)
#test_DataLoader 생성
test_DataLoader = DataLoader(test_data, batch_size=16, shuffle=True) 

### Predict 수행

In [17]:
#가장 확률이 큰 레이블들을 임시저장할 리스트
decoded = []
#gradient 연산을 끄기 위한 함수
with torch.no_grad(): 



    #test에는 레이블이 없고 차례대로 test사진을 뽑아준다.
    for i,images in enumerate(test_DataLoader):

        #test사진을 CUDA에 넣어준다.
        images = images.to('cuda')
        #학습 시킨 모델에 이미지 넣어주기
        logits = model(images)
        #softmax함수를 사용해서 확률을 뽑아준다.
        ps = F.softmax(logits, dim=1)
        #15개의 레이블에서 가장 확률이 큰 레이블을 가져온다.
        top_p, top_class = ps.topk(1, dim=1)
        #flatten data
        top_class = top_class.data.view(-1)
        #리스트에 예측한 레이블들을 저장
        decoded.append(top_class)
        print(top_class, top_class.shape)
  

        

In [ ]:
#epochs마다 tensor 타입으로 레이블들이 저장되기 때문에 이를 정수형으로 변환해서 리스트 pred에 저장해준다.
pred = []
for i in range(len(decoded)):
    for j in range(len(decoded[i])):
        pred.append(int(decoded[i][j]))
        
        
pred

예측 결과 인덱스를 저장한 pred 값을 사용해 클래스를 매칭하기 위한 class_name을 담아 놓은 리스트입니다.

주의할 점은 Train과정에서 사용된 class label 값과 같은 순서로 저장이 되어 있어야 한다는 점입니다


In [18]:
classes = [i.split('/')[-1] for i in glob.glob('../input/detect-the-bird/imgs/train/*')]

In [ ]:
id=[]
category =[]
for i in range(len(test_set)):
    id.append(test_set[i].split('/')[-1])
    category.append(classes[pred[i]])
pd.DataFrame({'Id':id,'Category':category}).to_csv('submission.csv',index=False)

---